In [44]:
import pandas as pd
import altair as alt
import urllib.request
import ssl
from scipy import stats

In [45]:
def load_data():
    return pd.read_csv("data/social_capital_zip_coords.csv")

df = load_data()

df.head()

df = df.drop('county', axis=1)
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('index', axis=1)

df['zip'] = df['zip'].astype(str)

df = df.set_index('zip')

df = df.dropna()

df.head()

,num_below_p50,pop2018,ec_zip,ec_se_zip,nbhd_ec_zip,ec_grp_mem_zip,ec_high_zip,ec_high_se_zip,nbhd_ec_high_zip,ec_grp_mem_high_zip,...,bias_grp_mem_zip,bias_grp_mem_high_zip,nbhd_bias_zip,nbhd_bias_high_zip,clustering_zip,support_ratio_zip,volunteering_rate_zip,civic_organizations_zip,lat,lng
zip,,,,,,,,,,,,,,,,,,,,,
10002,14952.651726,74993,0.86278,0.01468,0.96910,1.01334,1.65587,0.00994,1.65931,1.71111,...,0.19741,-0.06841,0.38141,-0.05916,0.08521,0.80943,0.06086,0.00923,40.715775,-73.986212
10003,1457.991344,54682,0.69477,0.01288,0.96008,1.06211,1.64008,0.00996,1.92181,1.74269,...,0.04605,-0.07615,0.28338,-0.43447,0.08443,0.67031,0.04932,0.00749,40.731829,-73.989181
10009,6397.973602,57925,0.92588,0.01689,1.02224,1.08775,1.68545,0.00851,1.74318,1.74940,...,0.17033,-0.07234,0.39303,-0.03504,0.09001,0.76925,0.06826,0.00784,40.726399,-73.978631
10011,1983.116590,50472,1.01316,0.02186,1.51580,1.24165,1.79870,0.00693,1.94907,1.85234,...,0.02655,-0.08069,0.17793,-0.05704,0.08488,0.77716,0.05453,0.01926,40.742039,-74.000620
10016,1311.327311,51057,1.05175,0.02065,1.76695,1.32880,1.76865,0.00703,1.93494,1.83606,...,-0.01056,-0.08028,0.03737,-0.05415,0.09728,0.84264,0.05273,0.00519,40.745224,-73.978297


In [46]:
# Save data from website as pandas df


from operator import index


ssl._create_default_https_context = ssl._create_unverified_context
url = urllib.request.urlopen('https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv')
city_state_df = pd.read_csv(url)

city_state_df.rename(columns={'zipcode': 'zip'}, inplace=True)

city_state_df.set_index('zip', inplace=True)

city_state_df.head()



# percentile = stats.percentileofscore()


,state_fips,state,state_abbr,county,city
zip,,,,,
35004,1,Alabama,AL,St. Clair,Acmar
35005,1,Alabama,AL,Jefferson,Adamsville
35006,1,Alabama,AL,Jefferson,Adger
35007,1,Alabama,AL,Shelby,Keystone
35010,1,Alabama,AL,Tallapoosa,New site


In [47]:
# Merge the two dataframes.  City, country, State, state_abbr, and state_fips are added as columns in the df.

df = pd.merge(df, city_state_df, on='zip')

print(df.columns)


Index(['num_below_p50', 'pop2018', 'ec_zip', 'ec_se_zip', 'nbhd_ec_zip',
       'ec_grp_mem_zip', 'ec_high_zip', 'ec_high_se_zip', 'nbhd_ec_high_zip',
       'ec_grp_mem_high_zip', 'exposure_grp_mem_zip',
       'exposure_grp_mem_high_zip', 'nbhd_exposure_zip', 'bias_grp_mem_zip',
       'bias_grp_mem_high_zip', 'nbhd_bias_zip', 'nbhd_bias_high_zip',
       'clustering_zip', 'support_ratio_zip', 'volunteering_rate_zip',
       'civic_organizations_zip', 'lat', 'lng', 'state_fips', 'state',
       'state_abbr', 'county', 'city'],
      dtype='object')


In [48]:
# Add a column for each percentile of each social capital dimension.

ec_zip_percentile_list = []

for i in df['ec_zip']:
    percentile = stats.percentileofscore(df['ec_zip'], i, kind='strict')
    percentile_val = round(percentile, 2)
    ec_zip_percentile_list.append(percentile_val)

df['ec_zip_percentile'] = ec_zip_percentile_list

clustering_zip_list = []

for i in df['clustering_zip']:
    percentile = stats.percentileofscore(df['clustering_zip'], i, kind='strict')
    percentile_val = round(percentile, 2)
    clustering_zip_list.append(percentile_val)

df['clustering_zip_percentile'] = clustering_zip_list

civic_organizations_list = []

for i in df['civic_organizations_zip']:
    percentile = stats.percentileofscore(df['civic_organizations_zip'], i, kind='strict')
    percentile_val = round(percentile, 2)
    civic_organizations_list.append(percentile_val)

df['civic_organizations_zip_percentile'] = civic_organizations_list

In [53]:
# Make 3 new columns that describe the meaning of each social capital metric. 
# The value of each percentile is the same for each row.

df['ec_zip_meaning'] = 'Baseline definition of economic connectedness: two times the share of high-SES friends among low-SES individuals, averaged over all low-SES individuals in the ZIP code.'

df['clustering_zip_meaning'] = 'The average fraction of an individual\s friend pairs who are also friends with each other.'

df['civic_organizations_zip_meaning'] = 'The number of Facebook Pages predicted to be “Public Good” pages based on page title, category, and other page characteristics, per 1,000 users in the ZIP code.'


,num_below_p50,pop2018,ec_zip,ec_se_zip,nbhd_ec_zip,ec_grp_mem_zip,ec_high_zip,ec_high_se_zip,nbhd_ec_high_zip,ec_grp_mem_high_zip,...,state,state_abbr,county,city,ec_zip_percentile,clustering_zip_percentile,civic_organizations_zip_percentile,ec_zip_meaning,clustering_zip_meaning,civic_organizations_zip_meaning
zip,,,,,,,,,,,,,,,,,,,,,
10002,14952.651726,74993,0.86278,0.01468,0.96910,1.01334,1.65587,0.00994,1.65931,1.71111,...,New york,NY,New York,New york,54.74,18.30,27.03,Baseline definition of economic connectedness:...,The average fraction of an individual\s friend...,The number of Facebook Pages predicted to be “...
10003,1457.991344,54682,0.69477,0.01288,0.96008,1.06211,1.64008,0.00996,1.92181,1.74269,...,New york,NY,New York,New york,23.71,16.70,17.57,Baseline definition of economic connectedness:...,The average fraction of an individual\s friend...,The number of Facebook Pages predicted to be “...
10009,6397.973602,57925,0.92588,0.01689,1.02224,1.08775,1.68545,0.00851,1.74318,1.74940,...,New york,NY,New York,New york,66.40,28.13,19.52,Baseline definition of economic connectedness:...,The average fraction of an individual\s friend...,The number of Facebook Pages predicted to be “...
10011,1983.116590,50472,1.01316,0.02186,1.51580,1.24165,1.79870,0.00693,1.94907,1.85234,...,New york,NY,New York,New york,79.68,17.63,74.39,Baseline definition of economic connectedness:...,The average fraction of an individual\s friend...,The number of Facebook Pages predicted to be “...
10016,1311.327311,51057,1.05175,0.02065,1.76695,1.32880,1.76865,0.00703,1.93494,1.83606,...,New york,NY,New York,New york,84.12,43.35,7.69,Baseline definition of economic connectedness:...,The average fraction of an individual\s friend...,The number of Facebook Pages predicted to be “...


In [61]:
# Remove unnecessary columns from the dataframe.

df = df.drop(columns=['exposure_grp_mem_zip','exposure_grp_mem_high_zip', 'nbhd_exposure_zip',
       'bias_grp_mem_zip', 'bias_grp_mem_high_zip', 'nbhd_bias_zip', 'nbhd_bias_high_zip',
       'support_ratio_zip', 'state_fips'])



Index(['num_below_p50', 'pop2018', 'ec_zip', 'nbhd_exposure_zip',
       'bias_grp_mem_zip', 'bias_grp_mem_high_zip', 'nbhd_bias_zip',
       'nbhd_bias_high_zip', 'clustering_zip', 'support_ratio_zip',
       'volunteering_rate_zip', 'civic_organizations_zip', 'lat', 'lng',
       'state_fips', 'state', 'state_abbr', 'county', 'city',
       'ec_zip_percentile', 'clustering_zip_percentile',
       'civic_organizations_zip_percentile', 'ec_zip_meaning',
       'clustering_zip_meaning', 'civic_organizations_zip_meaning'],
      dtype='object')


KeyError: "['exposure_grp_mem_zip', 'exposure_grp_mem_high_zip'] not found in axis"

In [16]:
sc_df = pd.read_csv("social_capital_zip.csv")
sc_df.reset_index(inplace=True)
sc_df.zip = sc_df.zip.astype(str)

sc_df.head()

percentile = stats.percentileofscore(sc_df['pop2018'], 17621, kind='strict')
print(round(percentile, 2))
sc_df.head()

# Drop all rows with NaN values to make processing easier.
sc_df = sc_df.dropna()

pop_percentile_list = []

for i in sc_df['pop2018']:
    percentile = stats.percentileofscore(sc_df['pop2018'], i, kind='strict')
    percentile_val = round(percentile, 2)
    pop_percentile_list.append(percentile_val)
    
sc_df['pop2018_percentile'] = pop_percentile_list


p50_percentile_list = []

for i in sc_df['num_below_p50'].dropna():
    percentile = stats.percentileofscore(sc_df['num_below_p50'], i, kind='strict')
    percentile_val = round(percentile, 2)
    p50_percentile_list.append(percentile_val)

sc_df['num_below_p50_percentile'] = p50_percentile_list

ec_zip_percentile_list = []

for i in sc_df['ec_zip'].dropna():
    percentile = stats.percentileofscore(sc_df['ec_zip'], i, kind='strict')
    percentile_val = round(percentile, 2)
    ec_zip_percentile_list.append(percentile_val)

sc_df['ec_zip_percentile'] = ec_zip_percentile_list

clustering_zip_list = []

for i in sc_df['clustering_zip']:
    percentile = stats.percentileofscore(sc_df['clustering_zip'], i, kind='strict')
    percentile_val = round(percentile, 2)
    clustering_zip_list.append(percentile_val)

sc_df['clustering_zip_percentile'] = clustering_zip_list

civic_organizations_list = []

for i in sc_df['civic_organizations_zip']:
    percentile = stats.percentileofscore(sc_df['civic_organizations_zip'], i, kind='strict')
    percentile_val = round(percentile, 2)
    civic_organizations_list.append(percentile_val)

sc_df['civic_organizations_zip_percentile'] = civic_organizations_list


sc_df.head()


71.16


,index,zip,county,num_below_p50,pop2018,ec_zip,ec_se_zip,nbhd_ec_zip,ec_grp_mem_zip,ec_high_zip,...,nbhd_bias_high_zip,clustering_zip,support_ratio_zip,volunteering_rate_zip,civic_organizations_zip,pop2018_percentile,num_below_p50_percentile,ec_zip_percentile,clustering_zip_percentile,civic_organizations_zip_percentile
0,0,1001,25013.0,995.787468,17621,0.88157,0.02422,1.51095,1.10210,1.47136,...,-0.21186,0.10572,0.94526,0.05650,0.01080,56.02,42.38,56.09,61.36,36.18
1,1,1002,25015.0,1312.117077,30066,1.18348,0.02227,0.97760,1.23333,1.62290,...,-0.24353,0.10340,0.90163,0.14951,0.03688,76.51,52.28,94.20,56.57,96.17
3,3,1005,25027.0,381.519745,4991,1.15543,0.03050,1.46491,1.30756,1.47733,...,-0.11397,0.10554,0.95837,0.15862,0.02163,19.01,11.54,92.33,61.01,80.99
4,4,1007,25015.0,915.396667,14967,1.19240,0.02046,1.17985,1.32294,1.56812,...,-0.21283,0.10391,0.94873,0.13053,0.01690,50.19,39.58,94.71,57.64,66.19
8,8,1013,25013.0,2616.550354,23065,0.69744,0.01274,0.53930,0.75807,1.23152,...,-0.40365,0.08648,0.89057,0.06191,0.00969,65.58,74.81,23.13,20.29,29.76
